In [ ]:
####THIS IS NOT UP TO DATE

In [10]:
import jax.numpy as np
from jax import grad, vmap
from jax.scipy.linalg import cho_factor,cho_solve
# Define parameters for the Gaussian function
alpha1 = 1.0
alpha2 = 1.0
alpha = 0.5

# Construct the A matrix
A = np.array([[alpha1 + alpha, -alpha, 0.0],
              [-alpha, alpha2 + alpha, -alpha],
              [0.0, -alpha, alpha1 + alpha]])

# Define the s vector (mean position terms) for Φ_i and Φ_j
r1 = np.array([1.0, 2.0, 3.0])  # Coordinates for r1 in 3D space
r2 = np.array([4.0, 5.0, 6.0])  # Coordinates for r2 in 3D space

# Define 3D mean vectors s1 and s2 for each Gaussian
s1 = np.array([0.5, 0.5, 0.5])  # Center of the Gaussian for r1
s2 = np.array([1.5, 1.5, 1.5])  # Center of the Gaussian for r2


# Define a constant term c if needed for the exponential (not shown in the initial code)
c = 0.0


In [11]:
def gaussian_wavefunction(r, A, s, c):
    # Compute the quadratic form: -1/2 * r^T A r
    quad_form = -0.5 * np.dot(r, np.dot(A, r))
    # Compute the linear term: r^T s
    linear_term = np.dot(r, s)
    # Combine terms for the Gaussian function
    return np.exp(quad_form + linear_term + c)


In [25]:
def kinetic_energy(psi_func, r):
    def second_derivative_along_dimension(i):
        # Define a function of a single variable along dimension i
        def single_dim_function(x):
            # Create a copy of r with the i-th component set to x
            r_modified = r.at[i].set(x)
            return psi_func(r_modified, A, s1, c)
        
        # Compute the second derivative with respect to the i-th component
        return grad(grad(single_dim_function))(r[i])

    # Sum the second derivatives across each dimension to get the Laplacian
    laplacian_r = np.sum([second_derivative_along_dimension(i) for i in range(3)])
    return -0.5 * laplacian_r


In [23]:
def kinetic_energy(psi_func, r):
    # Compute the Laplacian by taking the second derivative with respect to each dimension
    laplacian_r = np.sum([grad(grad(lambda x: psi_func(r.at[i].set(x), A, s1, c)))(r[i]) for i in range(3)])
    return -0.5 * laplacian_r


In [14]:
def A_chol_(A):
    A_pd, lower = cho_factor(A)
    return A_pd, lower

In [15]:
def potential_energy(r1, r2, psi_val):
    coulomb_term = 1 / np.linalg.norm(r1 - r2)
    nucleus_term_r1 = -9 / np.linalg.norm(r1)
    nucleus_term_r2 = -9 / np.linalg.norm(r2)
    return (coulomb_term + nucleus_term_r1 + nucleus_term_r2) * psi_val


In [16]:
def Hamiltonian(psi_func, r1, r2):
    # Evaluate the wavefunction
    psi_val = psi_func(r1, A, s1, c) * psi_func(r2, A, s2, c)
    return kinetic_energy(psi_func, r1) + kinetic_energy(psi_func, r2) + potential_energy(r1, r2, psi_val)


In [27]:
integration_limits = (-10.0, 10.0)
grid_size = 100
x = np.linspace(integration_limits[0], integration_limits[1], grid_size)

# Helper function to perform integration over a grid
def integrate_2d(f, x):
    dx = (integration_limits[1] - integration_limits[0]) / grid_size
    return np.sum(f * dx**2)

# Define matrix element computation for ⟨Φᵢ | Ĥ | Φⱼ⟩
def matrixElement_Phi_i_H_Phi_j():
    integrand = vmap(lambda x1, x2: gaussian_wavefunction(r1, A, s1, c) * Hamiltonian(gaussian_wavefunction, x1, x2))(x, x)
    return integrate_2d(integrand, x)

# Example calculation
print("Matrix Element ⟨Φᵢ | Ĥ | Φⱼ⟩ = ", matrixElement_Phi_i_H_Phi_j())


IndexError: Too many indices for array: 1 non-None/Ellipsis indices for dim 0.

In [ ]:
def calc_B_matrix(A,A_prime):
    return A+ A_prime

In [ ]:
def overlap(B, s, s_prime, c, c_prime, d=3):
    v = s+s_prime
    term1 = ((2*np.pi)**2 /( np.linalg.det(B)))**(d/2)
    term2 = np.exp(-0.5 * v.T @ B @ v + c + c_prime)
    return term1 @ term2

In [17]:
print("Matrix Element ⟨Φᵢ | Ĥ | Φⱼ⟩ = ", matrixElement_Phi_i_H_Phi_j())


Matrix Element ⟨Φᵢ | Ĥ | Φⱼ⟩ =  nan
